# LangGraph practice # 4


## 구성도

![4.practice.png](/home/ansgyqja/AI_application/images/4.practice.png)

In [ ]:
import os,sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('utils'), '..')))
from module.utils import * 
from module.prompt import * 
from module.custom_model import *

from typing import TypedDict, Annotated, List, Literal
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage,ToolMessage

In [ ]:
strt_langsmith('practice_4')

### 상태 클래스

In [ ]:
class State(TypedDict):
    messages : Annotated[list,add_messages]

### bind_tools 사용 여부에따른 답변 차이

1. no Instruction 

안녕하세요! 어떤 종류의 프롬프트 템플릿을 만들고 싶으신가요?

2. use Instruction 

안녕하세요! 어떤 종류의 프롬프트 템플릿을 만들고 싶으신가요? 

제가 도와드릴 수 있도록 프롬프트의 목적, 포함될 변수, 그리고 출력에 대한 제약 조건이나 요구 사항을 알려주세요.

- 해당 변수를 명확히 제시함 

- LangChain에서 도구 바인딩 시, LLM이 도구에 필요한 입력 정보를 어떤 형식으로 받아야 하는지 스키마로 알려주기 위한 역할

In [ ]:
# LLM에 대한 프롬프트 지침을 정의하는 데이터 모델
class Instructions(BaseModel):
    """Instructions on how to prompt the LLM."""
    
    objective: str =Field(description='What the objective of the prompt is') # 프롬프트의 목표
    
    variables: List[str] =Field(description='What variables will be passed into the prompt template') # 프롬프트 템플릿에 전달될 변수 목록
    
    constraints: List[str] = Field(description='Any constraints for what the output should NOT do')# 출력에서 피해야 할 제약 조건 목록

    requirements: List[str] = Field(description='Any requirements that the output MUST adhere to')# 출력이 반드시 따라야 할 요구 사항 목록

def gathering_infomation(state : State):
    
    question = state['messages']
    prompt = get_prompt_require_infomation() 
    llm = get_gemini()
    llm_with_tools =  llm.bind_tools([Instructions])
    chain = prompt | llm_with_tools 
    result = chain.invoke({"placeholder":question})
    return State({'messages':result})


def tool_response(state : State):
    """
        ToolMessage가 없으면 LLM은 도구 호출 결과를 알 수 없으므로, 도구 호출을 통해 얻은 정보를 반영한 후속 대화가 어려워집니다.
    """
    return State({'messages':[
            ToolMessage(content="Prompt generated!",tool_call_id=state["messages"][-1].tool_calls[0]["id"])  # 상태에서 도구 호출 ID를 가져와 메시지에 추가
        ]})


# 프롬프트 생성 체인 함수 정의
def prompt_generate(state):
    # 상태에서 프롬프트 메시지를 가져옴
    messages = get_prompt_messages(state["messages"])
    # LLM을 호출하여 응답 생성
    llm = get_gemini()
    response = llm.invoke(messages[0].content)
    # 생성된 응답을 메시지 목록으로 반환
    return {"messages": [response]}

### 노드 상태 조건

In [ ]:

# 상태 결정 함수 정의
# 상태에서 메시지 목록을 가져옴
def get_state(state)->Literal["tool_response",END,"gathering_infomation"]:
    """ 
        메시지 상태를 확인하여 tool_calls가 호출된 경우 다음 단계로 진행
    """
    messages = state["messages"]
    # 마지막 메시지가 AIMessage이고 도구 호출이 있는 경우
    if isinstance(messages[-1], AIMessage) and messages[-1].tool_calls:
        # 도구 메시지를 추가해야 하는 상태 반환
        return "tool_response"
    elif not isinstance(messages[-1], HumanMessage):
        # 대화 종료 상태 반환
        return END
    # 기본적으로 정보 수집 상태 반환
    return "gathering_infomation"

### 랭그래프 생성

In [ ]:
state_graph = StateGraph(State)
state_graph.add_node('gathering_infomation',gathering_infomation)
state_graph.add_node('tool_response',tool_response)
state_graph.add_node("prompt_generate", prompt_generate)

state_graph.add_edge(START,'gathering_infomation')
state_graph.add_conditional_edges(
    source='gathering_infomation',
    path=get_state
)
state_graph.add_edge("tool_response", "prompt_generate")
state_graph.add_edge("prompt_generate", END)

memory = get_check_pointer()
graph = state_graph.compile(checkpointer=memory)

# visualize_graph(graph)


### 시각화

In [ ]:
# visualize_graph(graph)
# print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	gathering_infomation(gathering_infomation)
	tool_response(tool_response)
	prompt_generate(prompt_generate)
	__end__([<p>__end__</p>]):::last
	__start__ --> gathering_infomation;
	gathering_infomation -.-> __end__;
	gathering_infomation -.-> tool_response;
	tool_response --> prompt_generate;
	prompt_generate --> __end__;
	gathering_infomation -.-> gathering_infomation;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



### 부분 테스트

In [ ]:
# query = "로보아르테 장애대응, 고객 질문 답변등 수행 챗봇 만들거고 변수는 question하나, 욕설은 절대안되고 , 친절한 말투 유지해줘! 추가 물어보지말고 "
# query = [HumanMessage(content='롸버트 치킨 고객 문의 응대, 정보 제공, 장애 지원 위한  상담 챗봇 프롬프트', additional_kwargs={}, response_metadata={}, id='305f750f-592c-4d3d-ba70-d2730553f5b4'), AIMessage(content='어떤 종류의 정보를 제공하고, 어떤 종류의 장애를 지원하는지에 대해 더 자세히 알려주시겠어요?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--a89d7fa2-b554-4710-ba80-2b96f53a98e9-0', usage_metadata={'input_tokens': 286, 'output_tokens': 25, 'total_tokens': 311, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='변수는 question과 answer 두개를 사용', additional_kwargs={}, response_metadata={}, id='9c37a943-5c88-48c9-a59b-f1473924d779'), AIMessage(content="알겠습니다. 'question'과 'answer' 두 개의 변수를 사용하시는군요.\n\n이제 다음 사항들을 명확히 할 수 있도록 도와주시겠어요?\n\n1.  **프롬프트의 구체적인 목표**: 챗봇이 고객 문의에 응대하고, 정보를 제공하며, 장애를 지원하는 것 외에, 이 프롬프트로 달성하고자 하는 가장 중요한 목표는 무엇인가요? (예: 특정 톤앤매너 유지, 문제 해결 시간 단축 등)\n2.  **출력에 대한 제약 조건**: 챗봇 응답이 절대로 해서는 안 되는 것이 있다면 알려주세요. (예: 개인 정보 언급 금지, 특정 단어 사용 금지 등)\n3.  **출력에 대한 요구 사항**: 챗봇 응답이 반드시 만족해야 하는 조건이 있다면 알려주세요. (예: 답변은 항상 명확하고 간결해야 함, 특정 형식 준수 등)", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--560bbbc9-cba5-4b0f-9319-3c5ad45e4765-0', usage_metadata={'input_tokens': 321, 'output_tokens': 203, 'total_tokens': 524, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='욕설은 절대 안되고 ', additional_kwargs={}, response_metadata={}, id='56caba3d-f3e7-4fcc-8614-4f22acf37e95'), AIMessage(content='알겠습니다. 욕설은 절대 사용하면 안 된다는 점을 명확히 했습니다.\n\n이제 마지막으로, 챗봇 응답이 반드시 만족해야 하는 요구 사항이 있다면 알려주세요. 예를 들어, 답변은 항상 명확하고 간결해야 한다거나, 특정 형식(예: 글머리 기호 사용)을 따라야 한다거나 하는 점이 있을 수 있습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--69c13e6d-41e2-4967-a8a0-f0a6bd96bf69-0', usage_metadata={'input_tokens': 533, 'output_tokens': 81, 'total_tokens': 614, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='필요시 rag에서 내용을 찾거나 아니면 너가 아는정보로 대답해줘', additional_kwargs={}, response_metadata={}, id='88abcf18-4d25-4d9a-96e1-7b4400ad44aa'), AIMessage(content='네, 알겠습니다. 사용자의 요청을 종합하면 다음과 같습니다.\n\n*   **목표**: 롸버트 치킨 고객 문의 응대, 정보 제공, 장애 지원을 위한 상담 챗봇 프롬프트 생성\n*   **변수**: `question`, `answer`\n*   **제약 조건**: 욕설 사용 금지\n*   **요구 사항**: 필요시 RAG(검색 증강 생성)를 통해 내용을 찾거나, 자체 지식으로 답변\n\n이 정보들을 바탕으로 프롬프트를 생성해도 될까요?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--d23f7d97-ede3-4ace-bdf5-ae86f584b61f-0', usage_metadata={'input_tokens': 636, 'output_tokens': 120, 'total_tokens': 756, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='ㅇㅇ', additional_kwargs={}, response_metadata={}, id='531cc9a8-de11-4f64-8c8d-6fb850abbb9a'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'Instructions', 'arguments': '{"variables": ["question", "answer"], "constraints": ["Do not use any profanity."], "requirements": ["If necessary, find information from RAG or answer using your own knowledge."], "objective": "Respond to customer inquiries, provide information, and offer support for disabilities for Robert Chicken, acting as a consultation chatbot."}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--b754a015-e3ec-4738-8070-5cb38bff87de-0', tool_calls=[{'name': 'Instructions', 'args': {'variables': ['question', 'answer'], 'constraints': ['Do not use any profanity.'], 'requirements': ['If necessary, find information from RAG or answer using your own knowledge.'], 'objective': 'Respond to customer inquiries, provide information, and offer support for disabilities for Robert Chicken, acting as a consultation chatbot.'}, 'id': 'a2b94a73-f8a6-48ca-9132-cb1cdfc594a0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 760, 'output_tokens': 70, 'total_tokens': 830, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Prompt generated!', id='e54c1290-d3d9-4527-af02-132455724572', tool_call_id='a2b94a73-f8a6-48ca-9132-cb1cdfc594a0')]
# messages = [SystemMessage(content='Given a task description or existing prompt, produce a detailed system prompt to guide a language model in completing the task effectively.\n\n# Guidelines\n\n- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.\n- Minimal Changes: If an existing prompt is provided, improve it only if it\'s simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.\n- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!\n    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.\n    - Conclusion, classifications, or results should ALWAYS appear last.\n- Examples: Include high-quality examples if helpful, using placeholders [in brackets] for complex elements.\n   - What kinds of examples may need to be included, how many, and whether they are complex enough to benefit from placeholders.\n- Clarity and Conciseness: Use clear, specific language. Avoid unnecessary instructions or bland statements.\n- Formatting: Use markdown features for readability. DO NOT USE ``` CODE BLOCKS UNLESS SPECIFICALLY REQUESTED.\n- Preserve User Content: If the input task or prompt includes extensive guidelines or examples, preserve them entirely, or as closely as possible. If they are vague, consider breaking down into sub-steps. Keep any details, guidelines, examples, variables, or placeholders provided by the user.\n- Constants: DO include constants in the prompt, as they are not susceptible to prompt injection. Such as guides, rubrics, and examples.\n- Output Format: Explicitly the most appropriate output format, in detail. This should include length and syntax (e.g. short sentence, paragraph, JSON, etc.)\n    - For tasks outputting well-defined or structured data (classification, JSON, etc.) bias toward outputting a JSON.\n    - JSON should never be wrapped in code blocks (```) unless explicitly requested.\n\nThe final prompt you output should adhere to the following structure below. Do not include any additional commentary, only output the completed system prompt. SPECIFICALLY, do not include any additional messages at the start or end of the prompt. (e.g. no "---")\n\n[Concise instruction describing the task - this should be the first line in the prompt, no section header]\n\n[Additional details as needed.]\n\n[Optional sections with headings or bullet points for detailed steps.]\n\n# Steps [optional]\n\n[optional: a detailed breakdown of the steps necessary to accomplish the task]\n\n# Output Format\n\n[Specifically call out how the output should be formatted, be it response length, structure e.g. JSON, markdown, etc]\n\n[User given variables should be wrapped in {brackets}]\n\n<Question>\n{question}\n</Question>\n\n<Answer>\n{answer}\n</Answer>\n\n# Examples [optional]\n\n[Optional: 1-3 well-defined examples with placeholders if necessary. Clearly mark where examples start and end, and what the input and output are. User placeholders as necessary.]\n[If the examples are shorter than what a realistic example is expected to be, make a reference with () explaining how real examples should be longer / shorter / different. AND USE PLACEHOLDERS! ]\n\n# Notes [optional]\n\n[optional: edge cases, details, and an area to call or repeat out specific important considerations]\n\n# Based on the following requirements, write a good prompt template:\n\n{\'variables\': [\'question\', \'answer\'], \'constraints\': [\'Do not use any profanity.\'], \'requirements\': [\'If necessary, find information from RAG or answer using your own knowledge.\'], \'objective\': \'Respond to customer inquiries, provide information, and offer support for disabilities for Robert Chicken, acting as a consultation chatbot.\'}\n', additional_kwargs={}, response_metadata={})]
# question = {'messages':query}
# result = prompt_generate(question)
# print(result)

### config 생성

In [ ]:
uuid = get_random_uuid()
config = get_runnable_config(recursion_limit=10, thread_id=uuid)

### input 창 활성화 통한 테스트 수행

In [ ]:
while True:
    try:
        # 사용자 입력 받기
        inputs = input("User (q/Q to quit): ")
    except:
        pass

    # 사용자 입력 출력
    print(f"\n\033[93m[사용자] {inputs}\033[0m")

    # 'q' 또는 'Q' 입력 시 루프 종료
    if inputs in {"q", "Q"}:
        print("AI: 다음에 또 만나요!")
        break

    # print(graph.invoke({'messages':inputs},config,stream_mode='values')) ## tool_calls

    # for g in graph.stream({'messages':inputs},config,stream_mode='values'):
    #     print(g)
    stream_graph(
        graph,
        {"messages": [HumanMessage(content=inputs)]},
        config=config,
    )

"""
롸버트 치킨 고객 문의 응대, 정보 제공, 장애 지원 위한  상담 챗봇 프롬프트
변수는 question과 answer 두개를 사용
욕설은 절대 안되고 
필요시 rag에서 내용을 찾거나 아니면 너가 아는정보로 대답해줘
"""
